In [1]:
import mne 
import os
from mne.io import  read_raw_edf
import pandas as pd



In [2]:
# Set the logging level to suppress messages
mne.set_log_level("ERROR")  # Set the level to ERROR to suppress most messages

# Set the path to the root folder containing patient folders
root_folder = '/Users/Samet/Documents/Bildung/BootcampSpiced/github_rice_regression/Capstone_project/chb-mit-scalp-eeg-database-1.0.0/'

# Create an empty list to store dataframes for each patient
patient_dataframes = []

# Iterate through patient folders
for patient_folder in os.listdir(root_folder):
    patient_path = os.path.join(root_folder, patient_folder)

    # Check if the item in the root folder is a directory (patient folder)
    if os.path.isdir(patient_path):
        #print(f"Processing patient folder: {patient_folder}")

        # Create an empty list to store dataframes for each .edf file in the current patient folder
        edf_dataframes = []

        # Get a list of .edf files sorted by filename numbers
        edf_files = sorted([f for f in os.listdir(patient_path) if f.endswith('.edf')])

        # Loop through the sorted .edf files
        for file in edf_files:
            file_path = os.path.join(patient_path, file)
            raw = mne.io.read_raw_edf(file_path, preload=True)

            # Downsample the raw data to sfreq=128
            raw.resample(128)

            # Convert the raw data to a Pandas DataFrame
            df = raw.to_data_frame()

            # Add a new column to the DataFrame to indicate the source file name
            df['File Name'] = file

            edf_dataframes.append(df)

        # Concatenate the dataframes for each .edf file for the current patient
        patient_dataframe = pd.concat(edf_dataframes)

        # Append the patient's dataframe to the list
        patient_dataframes.append(patient_dataframe)

# Now, 'patient_dataframes' is a list containing Pandas DataFrames for each patient with downsampled EEG data.


: 

In [3]:
# Set the path to the root folder containing patient folders
root_folder = '/Users/Samet/Documents/Bildung/BootcampSpiced/github_rice_regression/Capstone_project/chb-mit-scalp-eeg-database-1.0.0/'

# Create an empty list to store dataframes for each patient
patient_dataframes = []

# Iterate through patient folders
for patient_folder in os.listdir(root_folder):
    patient_path = os.path.join(root_folder, patient_folder)

    # Check if the item in the root folder is a directory (patient folder)
    if os.path.isdir(patient_path):
        # Create an empty list to store raw data
        patient_raws = []

        # Create an empty list to store file names
        file_names = []

        # Get a list of .edf files sorted by filename numbers
        edf_files = sorted([f for f in os.listdir(patient_folder) if f.endswith('.edf')])

        # Read the first .edf file to obtain the channel information
        first_file_path = os.path.join(patient_folder, edf_files[0])
        first_raw = mne.io.read_raw_edf(first_file_path, preload=True)

        # Loop through the sorted .edf files
        for file in edf_files:
            file_path = os.path.join(patient_folder, file)
            raw = mne.io.read_raw_edf(file_path, preload=True)
            
            raw_downsampled = raw.copy().resample(sfreq=128)
            

            # Set the channel information for each Raw object to match the first Raw object
            raw_downsampled.info = first_raw.info

            patient_raws.append(raw_downsampled)
            file_names.extend([file] * len(raw_downsampled.times))

        # Concatenate the raw objects into a single patient_combined object while maintaining a continuous time axis
        patient_combined = mne.concatenate_raws(patient_raws, preload=True)


        # Print info about the combined raw object for this patient
        print(patient_combined.info)

# You now have the EEG data from the .edf files in the patient folder in one patient_combined object with a continuous time axis.


FileNotFoundError: [Errno 2] No such file or directory: 'chb11'

In [14]:
df = patient_combined.to_data_frame()
# Add a new column to the DataFrame to indicate the source file name
df['File Name'] = file_names

In [15]:
df.shape

(18686464, 25)

In [16]:
type(df)

pandas.core.frame.DataFrame

In [18]:
df.head()

,time,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,...,T8-P8-0,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1,File Name
0,0.000000,-145.889824,-104.437950,-42.712763,-33.231272,-170.898152,-110.662484,11.982817,-56.693821,-139.224010,...,44.764205,74.665696,-106.011537,84.636952,43.103484,-57.184707,-264.745427,94.719394,44.764205,chb01_01.edf
1,0.003906,19.998681,14.586589,5.942093,5.114176,23.455757,15.226859,-1.495469,8.265635,19.049821,...,-5.903085,-11.423631,14.619662,-11.367358,-5.551373,7.423304,37.449772,-12.575222,-5.903085,chb01_01.edf
2,0.007812,-10.400608,-7.840311,-2.702939,-2.982295,-12.495403,-8.041283,1.278082,-4.729659,-9.780279,...,3.634643,8.669002,-7.695286,6.598238,3.093659,-2.615871,-22.604027,6.888621,3.634643,chb01_01.edf
3,0.011719,7.521448,2.545940,4.279262,1.486770,7.968614,5.961491,-0.573553,3.144477,7.997345,...,-2.189232,2.133745,4.962829,-4.438937,-3.888542,5.468633,12.280984,-4.141289,-2.189232,chb01_01.edf
4,0.015625,-6.548636,-2.030466,-4.010286,0.332089,-6.684477,-4.638781,-0.011514,-1.049334,-5.965863,...,2.918519,-8.928768,-4.007684,3.505916,4.401006,-7.245550,-5.458360,4.351446,2.918519,chb01_01.edf
